In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re

In [2]:
df1= pd.read_csv("project2_shopee_final.csv",index_col=0, encoding="utf-8").astype(str)

In [3]:
df1.head(2)

,product_id,product_name,category,sub_category,link,image,price,rating,description
0,231475,Bộ Quần Áo Thể Thao Thời Trang Nam Q5SHOP Form...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/B%E1%BB%99-Qu%E1%BA%A7n-%C3%...,https://cf.shopee.vn/file/e96fe533edf419d544c7...,79000.0,4.9,giới thiệu quần áo kết hình ảnh chụp không đền...
1,232886,SÉT ĐỒ THỂ THAO NAM🚗 MẪU MONTER ĐỘC ĐÁO HÀNG B...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/S%C3%89T-%C4%90%E1%BB%92-TH%...,https://cf.shopee.vn/file/b76fe4944cb0b0e6bb96...,59000.0,4.7,quần áo thể dãn mùa hè chất lạnh mẫu bảng sét ...


In [4]:
df = pd.DataFrame(df1, columns =['description'])

In [5]:
df.head()

,description
0,giới thiệu quần áo kết hình ảnh chụp không đền...
1,quần áo thể dãn mùa hè chất lạnh mẫu bảng sét ...
2,ớng cân cân chất liệu vải mè thái giãn thoải m...
3,thị trường sỉ chào quý thông sản phẩm lưu thiế...
4,sản phẩm thể nam thể nam thiết kế form chuẩn n...


In [6]:
# bug_ can not decode with float, should include .astype(str) when read csv file
# word_tokenize
df["description_wt"]=df['description'].apply(lambda x: word_tokenize(x, format="text"))

In [7]:
df.head(6)

,description,description_wt
0,giới thiệu quần áo kết hình ảnh chụp không đền...,giới_thiệu quần_áo kết hình_ảnh chụp không đền...
1,quần áo thể dãn mùa hè chất lạnh mẫu bảng sét ...,quần_áo thể dãn mùa hè chất lạnh mẫu bảng sét ...
2,ớng cân cân chất liệu vải mè thái giãn thoải m...,ớng cân cân chất_liệu vải mè thái giãn thoải_m...
3,thị trường sỉ chào quý thông sản phẩm lưu thiế...,thị_trường sỉ chào quý thông sản_phẩm lưu thiế...
4,sản phẩm thể nam thể nam thiết kế form chuẩn n...,sản_phẩm thể nam thể nam thiết_kế form chuẩn n...
5,đôi nam nữ mặt trăng mặt trời chất cotton mềm ...,đôi nam_nữ mặt_trăng mặt_trời chất cotton mềm ...


In [8]:
# sent_tokenize
#products_sent = df["products"].apply(lambda x: sent_tokenize(x))

In [9]:
#products_sent

# Gemsim
- https://pypi.org/project/gensim/
#### Reference: 
- link https://www.tutorialspoint.com/gensim/index.htm 
- link https://www.machinelearningplus.com/nlp/gensim-tutorial/ 

In [10]:
# Tokenize(split) the sentences into words
products_gem = [[text for text in x.split()] for x in df.description_wt]

In [11]:
len(products_gem)

17771

In [12]:
products_gem[:5]

[['giới_thiệu',
  'quần_áo',
  'kết',
  'hình_ảnh',
  'chụp',
  'không',
  'đền',
  'tiền',
  'gấp',
  'hình',
  'hoàn',
  'tiền',
  'sản_phẩm',
  'không',
  'mô_tả',
  'kết',
  'hàng',
  'mặc',
  'không',
  'màu',
  'không',
  'xù',
  'lông',
  'giặt',
  'máy',
  'thoải_mái',
  'freeship',
  'tất',
  'đơn',
  'hàng',
  'không',
  'hàng',
  'hà',
  'nội',
  'freeship',
  'không',
  'liên_hệ',
  'hướng',
  'mặc',
  'tập',
  'gái',
  'học',
  'chất_liệu',
  'vải',
  'áo_quần',
  'vải',
  'cotton',
  'thoáng',
  'mát',
  'túi',
  'túi',
  'thiết_kế',
  'đẹp_mắt',
  'chống',
  'đồ',
  'thấm',
  'hút',
  'mồ_hôi',
  'thoáng',
  'mát',
  'dãn',
  'tốt',
  'set',
  'sản_phẩm',
  'quần_đùi',
  'áo_cộc',
  'bảng',
  'lựa',
  'bảng',
  'thông',
  'thể'],
 ['quần_áo',
  'thể',
  'dãn',
  'mùa',
  'hè',
  'chất',
  'lạnh',
  'mẫu',
  'bảng',
  'sét',
  'quần_áo',
  'thể',
  'dãn',
  'mùa',
  'hè',
  'tiết',
  'sản_phẩm_chất_liệu',
  'lạnh',
  'chiều',
  'mềm',
  'mịn',
  'thấm_thoát',
  'mồ',
  'h

In [13]:
products_gem[:1]

[['giới_thiệu',
  'quần_áo',
  'kết',
  'hình_ảnh',
  'chụp',
  'không',
  'đền',
  'tiền',
  'gấp',
  'hình',
  'hoàn',
  'tiền',
  'sản_phẩm',
  'không',
  'mô_tả',
  'kết',
  'hàng',
  'mặc',
  'không',
  'màu',
  'không',
  'xù',
  'lông',
  'giặt',
  'máy',
  'thoải_mái',
  'freeship',
  'tất',
  'đơn',
  'hàng',
  'không',
  'hàng',
  'hà',
  'nội',
  'freeship',
  'không',
  'liên_hệ',
  'hướng',
  'mặc',
  'tập',
  'gái',
  'học',
  'chất_liệu',
  'vải',
  'áo_quần',
  'vải',
  'cotton',
  'thoáng',
  'mát',
  'túi',
  'túi',
  'thiết_kế',
  'đẹp_mắt',
  'chống',
  'đồ',
  'thấm',
  'hút',
  'mồ_hôi',
  'thoáng',
  'mát',
  'dãn',
  'tốt',
  'set',
  'sản_phẩm',
  'quần_đùi',
  'áo_cộc',
  'bảng',
  'lựa',
  'bảng',
  'thông',
  'thể']]

In [16]:
def gen_process(list):
    dictionary = corpora.Dictionary(list)
    dictionary.token2id
    corpus = [dictionary.doc2bow(text) for text in list]
    # Use TF-IDF Model to process corpus, obtaining index
    tfidf = models.TfidfModel(corpus) 
    
    # Numbers of features (word) in dictionary
    feature_cnt = len(dictionary.token2id)
    # Similarity calculation from sparseMatriy
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], 
                                            num_features = feature_cnt)
    
    return corpus, tfidf, index, dictionary

In [17]:
corpus, tfidf, index, dictionary = gen_process(products_gem)

In [18]:
data_result = pd.DataFrame(index[tfidf[corpus]])

In [19]:
data_result

,0,1,2,3,4,5,6,7,8,9,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
0,1.000000,0.144710,0.078223,0.055718,0.072788,0.046237,0.103197,0.035864,0.106959,0.0,...,0.053968,0.009577,0.014885,0.111793,0.155187,0.003397,0.010569,0.023133,0.143487,0.023317
1,0.144710,1.000000,0.045599,0.037887,0.114483,0.081348,0.122409,0.051671,0.058267,0.0,...,0.026900,0.001768,0.000000,0.049650,0.115119,0.000000,0.018360,0.029162,0.039254,0.055896
2,0.078223,0.045599,1.000000,0.043512,0.033888,0.072951,0.074996,0.024768,0.016666,0.0,...,0.033767,0.000000,0.000000,0.066965,0.063374,0.000000,0.022398,0.048207,0.051953,0.032276
3,0.055718,0.037887,0.043512,1.000000,0.059550,0.049593,0.274841,0.044598,0.092702,0.0,...,0.023130,0.062195,0.016125,0.055976,0.061608,0.000000,0.037077,0.019132,0.205439,0.036559
4,0.072788,0.114483,0.033888,0.059550,1.000000,0.049000,0.053498,0.026313,0.153682,0.0,...,0.026739,0.009333,0.005512,0.038240,0.070731,0.000000,0.027681,0.019152,0.060944,0.021961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17766,0.003397,0.000000,0.000000,0.000000,0.000000,0.003904,0.000000,0.002134,0.000000,0.0,...,0.000000,0.000000,0.005007,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
17767,0.010569,0.018360,0.022398,0.037077,0.027681,0.031057,0.063759,0.035023,0.027195,0.0,...,0.021674,0.000000,0.000000,0.089767,0.091503,0.000000,1.000000,0.002648,0.043852,0.020428
17768,0.023133,0.029162,0.048207,0.019132,0.019152,0.025983,0.037714,0.009141,0.021717,0.0,...,0.007321,0.020444,0.000000,0.623293,0.030775,0.000000,0.002648,1.000000,0.068746,0.053084
17769,0.143487,0.039254,0.051953,0.205439,0.060944,0.030039,0.220747,0.073441,0.079012,0.0,...,0.039219,0.041097,0.015214,0.177985,0.112567,0.000000,0.043852,0.068746,1.000000,0.023651


### Buil Recommdation - product based

In [20]:
### create sub_list
list_sub = df1['sub_category'].unique().tolist()
df1['sub_category_id'] = df1['sub_category'].map(lambda x: list_sub.index(x) +1)
df1['item_id'] = df1.apply(lambda x: int(str(x['product_id']) + str(x['sub_category_id']).zfill(2)), axis=1)
df1.head(3)

,product_id,product_name,category,sub_category,link,image,price,rating,description,sub_category_id,item_id
0,231475,Bộ Quần Áo Thể Thao Thời Trang Nam Q5SHOP Form...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/B%E1%BB%99-Qu%E1%BA%A7n-%C3%...,https://cf.shopee.vn/file/e96fe533edf419d544c7...,79000.0,4.9,giới thiệu quần áo kết hình ảnh chụp không đền...,1,23147501
1,232886,SÉT ĐỒ THỂ THAO NAM🚗 MẪU MONTER ĐỘC ĐÁO HÀNG B...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/S%C3%89T-%C4%90%E1%BB%92-TH%...,https://cf.shopee.vn/file/b76fe4944cb0b0e6bb96...,59000.0,4.7,quần áo thể dãn mùa hè chất lạnh mẫu bảng sét ...,1,23288601
2,231261,Đồ Bộ hè nam 🚚 FREESHIP 🚚 Quần áo thể thao nam...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/%C4%90%E1%BB%93-B%E1%BB%99-h...,https://cf.shopee.vn/file/b00adb38c9ec0584a434...,49000.0,4.7,ớng cân cân chất liệu vải mè thái giãn thoải m...,1,23126101


In [21]:
### create a a dict to get 5 similar products
results = {}

for idx, row in df1.iterrows():    
    similar_indices = data_result[idx].argsort()[-6:-1]
    # similar_items = [(data_result[idx][i]) for i in similar_indices]
    similar_items = [(data_result[idx][i], df1['item_id'][i]) for i in similar_indices]
    # print(similar_items[0:])
    results[row['item_id']] = similar_items[:]

In [22]:
results

{23147501: [(0.28313234, 1562409),
  (0.28524077, 1567709),
  (0.29158247, 22131617),
  (0.322315, 15126809),
  (0.35762104, 23288901)],
 23288601: [(0.5098354, 23317401),
  (0.59065086, 16226602),
  (0.6075898, 23463701),
  (0.91875774, 1663702),
  (0.99999994, 23322101)],
 23126101: [(0.35489842, 23122901),
  (0.357504, 23144001),
  (0.357504, 23182101),
  (0.37143928, 23358601),
  (0.86318654, 1629402)],
 23433301: [(0.98068196, 23110701),
  (0.98068196, 2324801),
  (0.9830329, 2320401),
  (1.0, 23312901),
  (1.0, 23433301)],
 23386701: [(0.40480524, 239901),
  (0.40559295, 232301),
  (0.4178987, 2571014),
  (0.44324726, 16145702),
  (0.447346, 1528809)],
 2353701: [(0.75211066, 2077713),
  (0.7602127, 1826016),
  (0.767357, 18126416),
  (0.768088, 18110016),
  (0.77062315, 18105516)],
 2321501: [(0.73888564, 23119401),
  (0.76486826, 23250001),
  (0.9041025, 23144901),
  (0.9709151, 2320801),
  (0.9792433, 2363901)],
 23276301: [(0.43601757, 2324101),
  (0.6360595, 23320701),
  (0.

In [23]:
### get product_name by input product_id
def item_name(product_id):
    return df1.loc[df1['item_id'] == product_id]['product_name'].to_list()[0].split('-')[0]

In [24]:
### get product recommended
def recommend(item_id, num):
    print('Recommend ' + str(num) + ' products similar to ' + item_name(item_id) + '....result below')
    print('-----')
    
    recs = results[item_id][:num]
    for rec in recs:
        print(rec[1])
        print('recommended product id:' + str(rec[1]) + ', ' + item_name(rec[1]) + '(score: )' + str(rec[0]) + ')')

In [25]:
#### try to recommend a product_id: 232886, with sub_id: 01, and 5 similar products
recommend(23288601,5)

Recommend 5 products similar to SÉT ĐỒ THỂ THAO NAM🚗 MẪU MONTER ĐỘC ĐÁO HÀNG BAO ĐẸP. FREEship....result below
-----
23317401
recommended product id:23317401, Sét Bộ Quần Áo Thể Thao Kẻ Sọc Cực Chất Cao Cấp, Áo Thun Cổ Tròn Ngắn Tay (score: )0.5098354)
16226602
recommended product id:16226602, [Có sẵn] Bộ Hè (score: )0.59065086)
23463701
recommended product id:23463701, Bộ thể thao nam hè NY phối nâu, Bộ nam thể thao hè NY thun lạnh không xù form dáng trẻ trung(score: )0.6075898)
1663702
recommended product id:1663702, BỘ thể thao nam nữ mẫu thun lạnh ETRENITEST mã ET(score: )0.91875774)
23322101
recommended product id:23322101, Nguyên bộ thể thao nam mẫu mới phong cách hàn quốc,🚛FREE SHIP🚛,(score: )0.99999994)


### Recommendation content based

In [26]:
### 
user_input = input('what are you looking for: ')

what are you looking for: áo Thun NGắn tay màu TRắng, vải Cotton


In [27]:
### clean user_input 
def pre_process(text):
    from underthesea import word_tokenize, pos_tag, sent_tokenize
    import re
    import regex
    
    user_input = re.sub('[\.\:\,\-\+\d\!\%\...\"\*\>\<\^\&\/\[\]\(\)\=\~\#]', ' ', text)
    user_input = regex.sub(r'\s+', ' ', user_input).strip()
    
    def loaddicchar():
            uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
            unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

            dic = {}
            char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
                '|')
            charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
                '|')
            for i in range(len(char1252)):
                dic[char1252[i]] = charutf8[i]
            return dic
         
        # 
    def covert_unicode(txt):
            dicchar = loaddicchar()
            return regex.sub(
                r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
                lambda x: dicchar[x.group()], txt)
                
    user_input = covert_unicode(user_input)
    user_input = word_tokenize(user_input, format="text")
    user_input = user_input.lower().split()
    
    return user_input

In [30]:
### funct to compute similarity
def user_input_process(list1,list2):
    list_process = pre_process(list1)
    corp, tf, idx, dicts = gen_process(list2)
    vec = dicts.doc2bow(list_process)
    sim = idx[tf[vec]]
    ### to dataframe and get indices
    df_user = pd.DataFrame(sim)
    df_top5 = df_user.sort_values(by=[0],ascending=False)[:5]
    list_index = df_top5.index.values.tolist()
    
    return list_index, list_process, df_top5

In [31]:
a, b, c = user_input_process(user_input,products_gem )

In [32]:
### print a list of similar index
a

[12005, 12261, 4769, 17502, 11933]

In [40]:
### print user_input after clean_process
print('user input: ',user_input)
print('After clean process: ', b)

user input:  áo Thun NGắn tay màu TRắng, vải Cotton
After clean process:  ['áo_thun', 'ngắn', 'tay', 'màu', 'trắng', 'vải', 'cotton']


In [35]:
### print top similar scores (top 5)
c

,0
12005,0.519179
12261,0.512660
4769,0.506929
17502,0.506929
11933,0.480981


In [36]:
### get row from a list index
df_recommend = df1.iloc[a]
df_recommend.reset_index(drop=True, inplace=True)

In [37]:
### return a dataframe of recommended items 
df_recommend

,product_id,product_name,category,sub_category,link,image,price,rating,description,sub_category_id,item_id
0,26579,"Đồ ngủ nam, tay ngắn, tất - Hàng nhập khẩu",Thời Trang Nam,Đồ Ngủ,https://shopee.vn/%C4%90%E1%BB%93-ng%E1%BB%A7-...,nan,607300.0,0.0,ên hệ hàng cổ áo cổ tròn áo kiểu tiết độ phù h...,12,2657912
1,261208,Bộ đồ ngủ pyjama ngắn tay thời trang Nam,Thời Trang Nam,Đồ Ngủ,https://shopee.vn/B%E1%BB%99-%C4%91%E1%BB%93-n...,nan,169000.0,0.0,đồ ngắn thời nam,12,26120812
2,241187,TAI NGHE Silver.crest,Thời Trang Nam,Đồ Hóa Trang,https://shopee.vn/TAI-NGHE-Silver.crest-i.1593...,https://cf.shopee.vn/file/ba857396a70961afe7dd...,323000.0,5.0,màu đen trắng màu đen trắng màu đen trắng màu ...,5,24118705
3,22670,Áo tê vịt quack quack đen - trắng,Thời Trang Nam,Áo Vest và Blazer,https://shopee.vn/%C3%81o-t%C3%AA-v%E1%BB%8Bt-...,nan,155000.0,5.0,màu đen trắng,17,2267017
4,261127,☃▦✳Bộ Đồ Ngủ 100% Cotton Tay Ngắn Phong Cách H...,Thời Trang Nam,Đồ Ngủ,https://shopee.vn/%E2%98%83%E2%96%A6%E2%9C%B3B...,https://cf.shopee.vn/file/e48626ec94d6b52b12d0...,573000.0,0.0,hàng tỉnh quảng đông vải cotton chất liệu vải ...,12,26112712
